# Gerenciamento de tráfego - parte 2

Nesta seção iremos abordar mais formas de gerenciar o tráfego com o Istio, até o momento vimos basicamente uma formas, o roteamento por nome do host (FQDN).

Nosso novo cenário a nossa aplicação irá expor os serviços de _login_ e _order_ e algumas das suas APIs para outras aplicações.

Não precisamos expor cada um dos serviços da nossa aplicação, podemos criar um ou mais _VirtualServices_ que representem esse cenário:

| url interna               | url externa                                  | service     | port |
| ---                       | ---                                          | ---         | ---  |
| http://front-end          | http://INGRESS_HOST:INGRESS_PORT/            | front-end   | 8000 |
|                           | http://INGRESS_HOST:INGRESS_PORT/front-end   |             |      |
| http://login              | http://INGRESS_HOST:INGRESS_PORT/login       | login       | 8000 |
| http://catalogue          | http://INGRESS_HOST:INGRESS_PORT/catalogue   | catalogue   | 8000 |
| http://orders             |                                              | orders      | 8000 |
| http://shipping           |                                              | shipping    | 8000 |
| http://cart               |                                              | cart        | 8000 |
| http://payment            |                                              | payment     | 8000 |
| http://accounts           |                                              | accounts    | 8000 |
| http://orders/db          |                                              | orders-db   | 8000 |
| http://queue              |                                              | queue       | 8000 |
| http://cart/db            |                                              | cart-db     | 8000 |
| http://accounts/db        |                                              | accounts-db | 8000 |

Mantenha um terminal aberto e o Kiali, iremos utilizá-los com frequência.

Vamos determinar a URI do Ingress Gateway e configurá-lo:

In [2]:
# Configurando acesso ao Ingress
export INGRESS_HOST=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.status.loadBalancer.ingress[0].ip}')
export INGRESS_PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name=="http2")].port}')
export SECURE_INGRESS_PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name=="https")].port}')
export TCP_INGRESS_PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name=="tcp")].port}')

echo "Ingress uri"
echo http://$INGRESS_HOST:$INGRESS_PORT
echo https://$INGRESS_HOST:$SECURE_INGRESS_PORT

Ingress uri
http://20.62.230.230:80
https://20.62.230.230:443


In [3]:
# Configurar um ingress gateway
kubectl apply -f exemplos/simul-shop/istio/10/default-gateway.yaml

gateway.networking.istio.io/default-gateway created


### Discussão: Versionamento de recursos web

Embora esteja fora do escopo desse curso, essa é uma discussão que em algum momento do desenvolvimento do sistema você terá que enfrentar, como indicar para os consumidores a versão da sua aplicação.

Podemos separar o tema em duas partes:

* Interface com usuário final
* APIs


#### Interface web

Não é interessante apresentar qualquer forma de versionamento para o usuário final, ou seja, não deveríamos expor qustões internas do sistemas, como portas, protocolos e versões para os usuários finais.

Também dificultaria testes A/B, implantações canário, ou qualquer forma de manipulação das requisições que necessitasse de transparência para o consumidor.

#### APIs

É uma estória completamente diferente, os consumidores destes recursos procuram algum tipo de garantia de que as versões que utilizam são estáveis, que o contrato que estabeleceram com o produtor não seja alterado para aquele ponto no tempo.

Existem prós e contras para cada escolha no forma de versionamento, entre elas:
    
* **Controle de versão por meio de caminho na URI**: http://www.simul-shop.com/orders/api/v1
    * Prós: os clientes podem armazenar recursos em cache facilmente
    * Contras: esta solução tem ramificações na base de código
* **Controle de versão por meio de parâmetros de consulta**: http://www.simul-shop.com/orders/api?version=v1
    * Prós: é uma maneira direta de criar uma versão de API e é fácil padronizar para a versão mais recente
    * Contras: os parâmetros de consulta são mais difíceis de usar para rotear solicitações para a versão adequada da API
* **Controle de versão por meio de cabeçalhos personalizados**: `curl -H “Accept-versions: v1” http://www.simul-shop.com/orders/api`
    * Prós: não confunde o URI com informações de versão
    * Contras: requer cabeçalhos personalizados
* **Controle de versão por meio de negociação de conteúdo**: `curl -H “Accept: application/vnd.xm.device+json; version=v1” http://www.simul-shop.com/orders/api`
    * Prós: nos permite criar versões de uma representação de recurso único em vez de criar versões de toda a API, o que nos dá um controle mais granular sobre as versões. Não requer a implementação de regras de roteamento de URI.
    * Contras: Exigir cabeçalhos HTTP com tipos de mídia torna mais difícil testar e explorar a API usando um navegador.

Do ponto de vista do design da APIs, incorporar a versão no URL não é uma prática recomendada porque resulta mudanças nas URLs, mesmo que o próprio recurso não tenha mudado.

[Roy Fielding](https://en.wikipedia.org/wiki/Roy_Fielding), um dos principais contribuidores para o protocolo HTTP e criador to REST, não recomenta utilizar qualquer forma de controle de versão para APIs ([Apresentação](https://www.slideshare.net/evolve_conference/201308-fielding-evolve)).

#### Por que não versionamento semântico?

De fato, a maioria das APIs utiliza o [versionamento semântico](https://semver.org/), mas não a versão completamente, pelo menos não na URI. Nada impediria que suas URL fossem `https://www.simul-shop.com/cart/api/1.0.2', porém o número da versão principal é o que você precisa para indicar aos seus consumidores a compatibilidade entre as versões. Os números que representam as versões menores e as correções não deveriam quebrar a compatibilidade e não justificariam a mudança da URL.

Se você precisa expressar a versão completa para seus consumidores, pode utilizar os cabeçalhos personalizados, como por exemplo o `Accept-versions` ou `Content-versions`.

#### Considerações

Em resumo, quando tratamos de interface web, geralmente procuramos ser o mais transparente o possível para o usuário final, mas se a questão é APIs, nossos consumidores, geralmente outros técnicos, necessitam de alguma garantia que as mesmas versões geram os mesmos resultados. Isso pode dificultar o uso de implementação canário por uma questão de semântica.

Felizmente, o Istio fornece recursos para lidar com todas as opções, então nós optamos por indicar a versão no cabeçalho, quando necessário.

Também optamos por não adicionar a palavra `api` na URL, por uma simples questão, ela não contribui para os nossos exemplos e você pode optar em criar seus serviços com interfaces web (micro front-ends) e programáticas (APIs), sendo assim, você poderia adicionar a palavra depois do roteamento do Istio, mas isso é com você.

## Rota baseada no caminho

![rota baseada no caminha](media/path-based-routes.png)

Vamos implementar a tabela acima, já vimos como configurar _VirtualService_ para hosts, vamos configurá-lo para caminho, a estrutura para as URIs do front-end


```yaml
spec:
  hosts:
    - "*"
  gateways:
  - default-gateway
  http:
  - match: # <-- Regra prefixo /catalogue
    - uri:
        prefix: /login
    rewrite:
      uri: /
    route:
    - destination:
        host: login # FQDN completo ou abreviado
        subset: v1
  - match: # <-- Regra prefixo /catalogue
    - uri:
        prefix: /catalogue
    rewrite:
      uri: /
    route:
    - destination:
        host: catalogue # FQDN completo ou abreviado
        subset: v1
  - route: # <-- Rota padrão
    - destination:
        host: front-end 
        subset: v1
```

**Dica pro**: Você pode configurar regras para vários serviços no mesmo VirtualService, porém, para a organização, governança e entrega dessas regras pode se tornar um desafio conforme o número de serviços cresce.

Uma boa prática é criar um arquivo de entrega para cada serviço, você pode manter as _DestinationRules_ no mesmo arquivo ou em arquivos separados. Nestes exemplos, manteremos as _DestinationRules_ e _VirtualServices_ no mesmo arquivo.

Vamos aplicar as configuração para os serviços de [login](exemplos/simul-shop/istio/11/login-dr-vs.yaml), [catalogue](exemplos/simul-shop/istio/11/catalogue-dr-vs.yaml) e [front-end](exemplos/simul-shop/istio/11/front-end-dr-vs.yaml).

In [43]:
# Login
# DestinationRules e VirtualServices
kubectl apply -f exemplos/simul-shop/istio/11/login-dr-vs.yaml

destinationrule.networking.istio.io/login unchanged
virtualservice.networking.istio.io/login unchanged


In [44]:
# Catalogue
# DestinationRules e VirtualServices
kubectl apply -f exemplos/simul-shop/istio/11/catalogue-dr-vs.yaml

destinationrule.networking.istio.io/catalogue unchanged
virtualservice.networking.istio.io/catalogue unchanged


In [45]:
# Front-end
# DestinationRules e VirtualServices
kubectl apply -f exemplos/simul-shop/istio/11/front-end-dr-vs.yaml

destinationrule.networking.istio.io/front-end unchanged
virtualservice.networking.istio.io/front-end configured


Vamos verificar o que foi criado:

In [46]:
kubectl get vs,dr

NAME                                           GATEWAYS              HOSTS   AGE
virtualservice.networking.istio.io/catalogue   ["default-gateway"]   ["*"]   3h30m
virtualservice.networking.istio.io/front-end   ["default-gateway"]   ["*"]   3h30m
virtualservice.networking.istio.io/login       ["default-gateway"]   ["*"]   3h30m

NAME                                            HOST        AGE
destinationrule.networking.istio.io/catalogue   catalogue   3h30m
destinationrule.networking.istio.io/front-end   front-end   3h30m
destinationrule.networking.istio.io/login       login       3h30m


Vamos testando as configurações:

In [7]:
# front-end - rota padrão
http -v "$INGRESS_HOST:$INGRESS_PORT/"

GET / HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0



HTTP/1.1 200 OK
content-length: 108
content-type: application/json
date: Wed, 23 Dec 2020 15:07:06 GMT
server: istio-envoy
x-envoy-upstream-service-time: 763

{
    "app": "front-end",
    "description": "Hi there!",
    "name": "greetings",
    "version": "v1",
    "when": "2020-12-23 15:07:07"
}




In [8]:
# front-end - regra match.uri
http -v "$INGRESS_HOST:$INGRESS_PORT/front-end"

GET /front-end HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0



HTTP/1.1 200 OK
content-length: 108
content-type: application/json
date: Wed, 23 Dec 2020 15:07:42 GMT
server: istio-envoy
x-envoy-upstream-service-time: 2

{
    "app": "front-end",
    "description": "Hi there!",
    "name": "greetings",
    "version": "v1",
    "when": "2020-12-23 15:07:42"
}




In [9]:
# Login - regra match.uri
http -v "$INGRESS_HOST:$INGRESS_PORT/login"

GET /login HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0



HTTP/1.1 200 OK
content-length: 104
content-type: application/json
date: Wed, 23 Dec 2020 15:07:48 GMT
server: istio-envoy
x-envoy-upstream-service-time: 14

{
    "app": "login",
    "description": "Hi there!",
    "name": "greetings",
    "version": "v1",
    "when": "2020-12-23 15:07:48"
}




In [10]:
# Catalogue - regra match.uri
http -v "$INGRESS_HOST:$INGRESS_PORT/catalogue"

GET /catalogue HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0



HTTP/1.1 200 OK
content-length: 108
content-type: application/json
date: Wed, 23 Dec 2020 15:08:07 GMT
server: istio-envoy
x-envoy-upstream-service-time: 530

{
    "app": "catalogue",
    "description": "Hi there!",
    "name": "greetings",
    "version": "v1",
    "when": "2020-12-23 15:08:07"
}




## Rota baseada no cabeçalho

Os _VirtualServices_ são uma das principais configurações do gerenciamento de tráfico do Istio, nesse exemplo iremos utilizar os dados do cabeçalho da requisição para decidir qual versão do serviço será servida para o requisitante.

Esse tipo de controle permite realizar testes e entregas para grupos de usuários. Diferente da entrega canário, que direciona o tráfego com base em percentuais, a rota baseada no cabeçalho, permite mirar grupos específicos.

Neste cenário os usuários que acessarem o front-end na região _Southeast_ serão direcionados para a versão 2 e os demais para a versão 1. 

Vamos aplica a configuração [front-end-route-header.yaml](exemplos/simul-shop/istio/11/route-header/front-end-route-header.yaml).

In [11]:
# Deployment da versão 2 do front-end
kubectl apply -f exemplos/simul-shop/manifests/8/front-end-deployment-v2.yaml

deployment.apps/front-end-v2 configured


In [12]:
# Configurar o VirtualService do front-end para direcionar com base nos campos do cabeçalho
kubectl apply -f exemplos/simul-shop/istio/11/front-end-route-header-vs.yaml

virtualservice.networking.istio.io/front-end configured


Nossa aplicação já pode ser acessada pela uri http://INGRESS_HOST:INGRESS_PORT/

In [13]:
# Rota padrão
http -v "$INGRESS_HOST:$INGRESS_PORT"

GET / HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0



HTTP/1.1 200 OK
content-length: 108
content-type: application/json
date: Wed, 23 Dec 2020 15:09:14 GMT
server: istio-envoy
x-envoy-upstream-service-time: 2

{
    "app": "front-end",
    "description": "Hi there!",
    "name": "greetings",
    "version": "v1",
    "when": "2020-12-23 15:09:15"
}




A versão acessada foi a v1, vamos adicionar o cabeçalho.

In [14]:
# Regra #1 - campo user-region do cabeçalho igual a Southeast
http -v "$INGRESS_HOST:$INGRESS_PORT" "user-region: Southeast"

GET / HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0
user-region: Southeast



HTTP/1.1 200 OK
content-length: 108
content-type: application/json
date: Wed, 23 Dec 2020 15:09:24 GMT
server: istio-envoy
x-envoy-upstream-service-time: 655

{
    "app": "front-end",
    "description": "Hi there!",
    "name": "greetings",
    "version": "v2",
    "when": "2020-12-23 15:09:24"
}




Com o campo no cabeçalho o serviço chamado foi o v2.

Você pode usar uma combinação de campos do cabeçalho e tirar proveito de campos [padrão](https://en.wikipedia.org/wiki/List_of_HTTP_header_fields) para guiar a lógica do seu serviço.

* content-length
* content-type
* date
* Host
* User-Agent

## Multiplas regras de tráfego

Como discutimos anteriormente, você pode combinar multiplas regras em um único _VirtualService_, inclusive de serviços diferentes, porém isso pode ficar confuso conforme o numero de serviços cresce, mas é uma boa prática, combinar as regras em um único _VirtualService_ para o mesmo serviço, dessa forma podemos avaliar o seu comportamento em um único lugar.

As regras são avaliadas na ordem, as rotas HTTP serão aplicadas às portas de serviço da plataforma chamadas 'http -' / 'http2 -' / 'grpc- *', portas de gateway com protocolo HTTP / HTTP2 / GRPC / TLS-terminated-HTTPS e portas de entrada de serviço usando HTTP / HTTP2 / Protocolos GRPC. A primeira regra que corresponder a solicitação de entrada será usada.

Neste cenário queremos combinar a uri com campos do cabeçalho, somente requisições com uri iniciando com `/front-end` e cabeçalho com o campo `user-region: Southeast` serão direcionados para a v2.

Nesse cenário hipotético, após os testes, iremos direcionar as requisições com prefixo `/front-end` também para a v2.

E como boa prática, se nenhuma das regras for satisfeita, a requisição será direcionada para a v1.

In [15]:
# Virtual Service
kubectl apply -f exemplos/simul-shop/istio/11/front-end-multi-route-vs.yaml

virtualservice.networking.istio.io/front-end configured


In [16]:
# Rota padrão - v1
http -v "$INGRESS_HOST:$INGRESS_PORT/" "user-region: Southeast"

GET / HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0
user-region: Southeast



HTTP/1.1 200 OK
content-length: 108
content-type: application/json
date: Wed, 23 Dec 2020 15:17:55 GMT
server: istio-envoy
x-envoy-upstream-service-time: 2

{
    "app": "front-end",
    "description": "Hi there!",
    "name": "greetings",
    "version": "v1",
    "when": "2020-12-23 15:17:55"
}




In [17]:
# Rota padrão - v1
http -v "$INGRESS_HOST:$INGRESS_PORT/"

GET / HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0



HTTP/1.1 200 OK
content-length: 108
content-type: application/json
date: Wed, 23 Dec 2020 15:18:00 GMT
server: istio-envoy
x-envoy-upstream-service-time: 2

{
    "app": "front-end",
    "description": "Hi there!",
    "name": "greetings",
    "version": "v1",
    "when": "2020-12-23 15:18:01"
}




In [18]:
# Regra #2: uri iniciando em /front-end - v1 (mas pode ser alterada para outra versão)
http -v "$INGRESS_HOST:$INGRESS_PORT/front-end"

GET /front-end HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0



HTTP/1.1 200 OK
content-length: 108
content-type: application/json
date: Wed, 23 Dec 2020 15:18:19 GMT
server: istio-envoy
x-envoy-upstream-service-time: 2

{
    "app": "front-end",
    "description": "Hi there!",
    "name": "greetings",
    "version": "v1",
    "when": "2020-12-23 15:18:19"
}




In [19]:
# Regra #1: user-regio=Southeast e uri iniciando em /front-end - v2
http -v "$INGRESS_HOST:$INGRESS_PORT/front-end" "user-region: Southeast"

GET /front-end HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0
user-region: Southeast



HTTP/1.1 200 OK
content-length: 108
content-type: application/json
date: Wed, 23 Dec 2020 15:18:28 GMT
server: istio-envoy
x-envoy-upstream-service-time: 2

{
    "app": "front-end",
    "description": "Hi there!",
    "name": "greetings",
    "version": "v2",
    "when": "2020-12-23 15:18:28"
}




In [20]:
# Rota padrão, mas não tem re-escrita da URI - 404
http -v "$INGRESS_HOST:$INGRESS_PORT/anything"

GET /anything HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0



HTTP/1.1 404 Not Found
content-length: 22
content-type: application/json
date: Wed, 23 Dec 2020 15:18:39 GMT
server: istio-envoy
x-envoy-upstream-service-time: 28

{
    "detail": "Not Found"
}




## Modificando os cabeçalhos de resposta

Os _VirtualServices_ podem adicionar ou remover campos do cabeçalho.

Neste exemplo, queremos que as requisições que não são originadas de _Southeast_ tenham o campo do cabeçalho `user-region` com o valor `other`. Esse dado pode ser utilizada para regras em outros serviços, ou para fins de log.

In [21]:
# VirtualService
kubectl apply -f exemplos/simul-shop/istio/11/front-end-change-header-vs.yaml

virtualservice.networking.istio.io/front-end configured


In [22]:
# Regra 1 - v2
http -v "$INGRESS_HOST:$INGRESS_PORT/" "user-region: Southeast"

GET / HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0
user-region: Southeast



HTTP/1.1 200 OK
content-length: 108
content-type: application/json
date: Wed, 23 Dec 2020 15:19:42 GMT
server: istio-envoy
x-envoy-upstream-service-time: 2

{
    "app": "front-end",
    "description": "Hi there!",
    "name": "greetings",
    "version": "v2",
    "when": "2020-12-23 15:19:43"
}




In [23]:
# Rota padrão - v1
http -v "$INGRESS_HOST:$INGRESS_PORT/"

GET / HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: 20.62.230.230:80
User-Agent: HTTPie/2.3.0



HTTP/1.1 200 OK
content-length: 108
content-type: application/json
date: Wed, 23 Dec 2020 15:19:50 GMT
server: istio-envoy
user-region: other
x-envoy-upstream-service-time: 2

{
    "app": "front-end",
    "description": "Hi there!",
    "name": "greetings",
    "version": "v1",
    "when": "2020-12-23 15:19:56"
}




Procure pelo campo `user-region` no cabeçalho. Todas as requisições que forem direcionadas pela rota padrão terão o campo `user-region: other` adicionado.

## Retentativas e timeouts

Arquiteturas distribuídas significa solicitações na rede, aumentando a chance de falhas temporárias como congestionamento da rede.

Adicionar políticas de repetição para solicitações e ezpiração do tempo de espera ajudam a construir serviços resilientes em uma arquitetura de serviços. Frequentemente, essa lógica de repetição é incorporada no serviço, mas com o Istio, você pode definir políticas de repetição com uma regra de tráfego, a serem executadas pelos sidecars, permitindo padronizar as políticas de forma independente dos  protocolos e linguagens de programação.

Uma configuração tentativas especifica o número máximo de vezes que um proxy Envoy tenta se conectar a um serviço se a chamada inicial falhar.

### _Timeout_

Nosso primeira [configuração](exemplos/simul-shop/istio/11/login-timeout-vs.yaml) será ajustar o tempo de espera (timeout) para 10s. Para isso vamos configurar o _VirtualService_ do login e executar o serviço passando diferntes tempos de atraso.

> Na seção Engenharia do Caos, veremos como fazer isso usando configurações do Istio.

In [107]:
# VirtualService
kubectl apply -f exemplos/simul-shop/istio/11/login-timeout-vs.yaml

virtualservice.networking.istio.io/login unchanged


Para testar vamos executar o serviço, mas antes, abra um terminal para monitorar os logs do serviço.

Em um terminal digite `stern -l app=login -c istio-proxy`

Agora vamos executar o serviço.

In [109]:
# Atraso de 5s a 15s, incremento de 2s
for i in $(seq 5 2 15);
    do kubectl exec -it svc/front-end -c front-end -- http -v "http://login:8000/r?code=200&wait=$i";
done

GET /r?code=200&wait=5 HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: login:8000
User-Agent: HTTPie/0.9.8



HTTP/1.1 200 OK
content-length: 110
content-type: application/json
date: Wed, 23 Dec 2020 21:13:55 GMT
server: envoy
x-envoy-upstream-service-time: 5007

{
    "detail": "At 2020-12-23 21:13:55 this request asks for 200 and waits for 5s and now is 2020-12-23 21:14:00."
}

GET /r?code=200&wait=7 HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: login:8000
User-Agent: HTTPie/0.9.8



HTTP/1.1 200 OK
content-length: 110
content-type: application/json
date: Wed, 23 Dec 2020 21:14:02 GMT
server: envoy
x-envoy-upstream-service-time: 7008

{
    "detail": "At 2020-12-23 21:14:03 this request asks for 200 and waits for 7s and now is 2020-12-23 21:14:10."
}

GET /r?code=200&wait=9 HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: login:8000
User-Agent: HTTPie/0.9.8



HTTP/1.1 200 OK
content-l

O tempo de espera (timeout) ajustado para 10 segundos permitiu que o serviço fosse executado para tempos de espera 5s, 7s e 9s. Além desse ponto, o serviço retornou _timeout_

### _Retries_

Neste cenário, simulamos um problema com o serviço de login. Modificaremos o _Deployment_ do `front-end` para pausar a chamada automática aos serviços, isoo dificultaria acompanhar os logs.

In [110]:
# Modificar o _VirtualService_ do front-end
kubectl apply -f exemplos/simul-shop/manifests/8/front-end-deployment-no-auto.yaml

deployment.apps/front-end-v1 configured
deployment.apps/front-end-v2 created


In [129]:
# VirtualService
kubectl apply -f exemplos/simul-shop/istio/11/login-retry-vs.yaml

virtualservice.networking.istio.io/login configured


Como os tempos limite, o comportamento de retentativa pode não atender às necessidades do seu aplicativo em termos de latência (muitas tentativas para um serviço com falha podem tornar as coisas piores). Você pode ajustar as configurações por serviço.

Para configurar o comportamento das retentativa, pode-se adicionar tempos limite por nova tentativa, especificando a quantidade de tempo que você deseja esperar para cada tentativa.

Nesta [configuração](exemplos/simul-shop/istio/11/login-retry-vs.yaml) serão 3 tentativas após uma falha inicial, cada uma com um tempo limite de 2 segundos.

Acompanhe o log, você verá quatro entradas, a inicial e mais três, uma vez que nosso serviçp continuará a restornar o erro em todas as tentativas.

In [162]:
kubectl exec -it svc/front-end -c front-end -- bash -c 'time http -v "http://login:8000/r?code=504&wait=1"'

GET /r?code=504&wait=1 HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: login:8000
User-Agent: HTTPie/0.9.8



HTTP/1.1 504 Gateway Timeout
content-length: 110
content-type: application/json
date: Wed, 23 Dec 2020 22:21:27 GMT
server: envoy
x-envoy-upstream-service-time: 4067

{
    "detail": "At 2020-12-23 22:21:31 this request asks for 504 and waits for 1s and now is 2020-12-23 22:21:32."
}


real	0m4.284s
user	0m0.197s
sys	0m0.017s


O tempo total de execução foi 4x o tempo do serviço.

E se o serviço retornar um código 200 em qualquer uma das três tentativas, a requisição encerrará com sucesso e o tempo total de execução será a soma das tentativas.

In [159]:
kubectl exec -it svc/front-end -c front-end -- bash -c 'time http -v "http://login:8000/r?code=200&wait=1"'

GET /r?code=200&wait=1 HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate
Connection: keep-alive
Host: login:8000
User-Agent: HTTPie/0.9.8



HTTP/1.1 200 OK
content-length: 110
content-type: application/json
date: Wed, 23 Dec 2020 22:15:09 GMT
server: envoy
x-envoy-upstream-service-time: 1003

{
    "detail": "At 2020-12-23 22:15:10 this request asks for 200 and waits for 1s and now is 2020-12-23 22:15:11."
}


real	0m1.227s
user	0m0.172s
sys	0m0.050s


Nesta última execução, o serviço retorna após 1s com sucesso (200).

Você deverá ver os errors no kiali e jaeger. No jaeger filtre os rastros colocando em _Service_ escolha `front-end.default` e em _Tags_ preencha com o valor `http.status_code=500`

![Jaeger search retries](media/jaeger-retries-search.png)

E as execuções que acompanhamos nos logs.

![Jaeger failed retries](media/jaeger-failed-retries.png)

## Interrupção de circuíto

TODO

## Espelhando o tráfego

TODO